In [1]:
!pip install tensorflow
import os
import logging
from imageio import imread
import matplotlib
from matplotlib import pyplot as plt

import morphsnakes as ms

In [2]:
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import convolve
import os
from scipy import ndimage
import SimpleITK as sitk
import sys


**3D data and manual mask**

In [63]:
IMDIR = './debri_data/'
segDir = './Segmentation.seg.nrrd'

**Reading IRM**


In [138]:
print("Reading Dicom directory:", IMDIR)
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(IMDIR)
reader.SetFileNames(dicom_names)

image = reader.Execute()

taille = image.GetSize()
print("Image size:", taille[0], taille[1], taille[2])

if "SITK_NOSHOW" not in os.environ:
    sitk.Show(image, "Dicom Series")



Reading Dicom directory: ./debri_data/
Image size: 256 256 36


**Preparing 3d data and manual mask**

In [157]:
##manual mask
mask_sitk = sitk.ReadImage(segDir)
mask = sitk.GetArrayFromImage(mask_sitk)
mask_pred = mask[..., 0][0:36,0:256,0:256]
mask_pred = mask_pred.astype('float32')
mask_pred = np.where(mask_pred>0, 1.0, 0)
mask_sitk = sitk.GetImageFromArray(mask_pred)

print("manual mask shape: ",mask_pred.shape)

##3d data
V = sitk.GetArrayFromImage(image)
V = V.astype('float32')
print("3D data shape: ", np.shape(V))

##visualize manual mask
sitk.Show(mask_sitk, "Dicom Series")

manual mask shape:  (36, 256, 256)
3D data shape:  (36, 256, 256)


##  Sheet-like filter

**Function to compute the hessian eigenvalues**

In [126]:
from warnings import warn
import numpy as np
from scipy import linalg
from skimage.feature import hessian_matrix, hessian_matrix_eigvals

def compute_hessian_eigenvalues(image, sigma, sorting='none',
                                mode='constant', cval=0,
                                use_gaussian_derivatives=False):

    # Convert image to float
    image = image.astype('float32')
    # rescales integer images to [0, 1]
    image /= 255.0

    # Make nD hessian
    hessian_matrix_kwargs = dict(
        sigma=sigma, order='rc', mode=mode, cval=cval,
    )
    hessian_elements = hessian_matrix(image, **hessian_matrix_kwargs)
    if not use_gaussian_derivatives:
        # Kept to preserve legacy behavior
        hessian_elements = [(sigma ** 2) * e for e in hessian_elements]

    # Compute Hessian eigenvalues
    hessian_eigenvalues = hessian_matrix_eigvals(hessian_elements)

    if sorting == 'abs':

        # Sort eigenvalues by absolute values in ascending order
        hessian_eigenvalues = np.take_along_axis(
            hessian_eigenvalues, abs(hessian_eigenvalues).argsort(0)[::-1], 0)

    elif sorting == 'val':

        # Sort eigenvalues by values in descending order
        hessian_eigenvalues = np.sort(abs(hessian_eigenvalues), axis=0, reverse=True)

    # Return Hessian eigenvalues
    return hessian_eigenvalues

**Computing hessian matrix for our data**

In [158]:
sigma = 1.0
H = compute_hessian_eigenvalues(V, sigma, sorting='abs',
                                mode='constant', cval=0,
                                use_gaussian_derivatives=False)

print("Hessian matrix's shape: ",H.T.shape) ##each 'cube' contains an array of the 3 eigenvalues of the corresponding voxel

Hessian matrix's shape:  (256, 256, 36, 3)


**Sheetnees filter**

In [129]:
def sheetness_filter(image, eigvals, alpha, thresh):

    s = np.zeros(np.shape(image))
    
    lambda1 = np.maximum(eigvals[0], np.finfo(np.float32).eps)
    lambda2, lambda3 = np.maximum(eigvals[1:], np.finfo(np.float32).eps)
    
    r_sheet = lambda2 / lambda1
    r_noise = np.sqrt(lambda1**2 + lambda2**2 +lambda3**2) 
    beta = np.amax(r_noise)/20
    
    s = 1.0 - np.exp(-r_noise**2 / (2 * beta**2)) 
    s *= np.exp(-r_sheet**2 / (2 * alpha**2))
    
    lambda1 = eigvals[0]
    s = np.where(lambda1==0, 0, s)
    
    mask = np.where(s>thresh, 1, 0)
    im_masked = V * mask
    return s, im_masked 


**Applying sheetness filter and generating the 1st mask**

In [159]:
import skimage
from skimage.feature import canny
from scipy import ndimage as ndi

eigvals = H
alpha = 0.5
thresh = 0.55
s, image_masked = sheetness_filter(V, eigvals, alpha, thresh)
labeled_image, count = skimage.measure.label(image_masked,
                                             connectivity=2, return_num=True)
labeled_image =labeled_image.astype('float32')
labeled_image = np.where(labeled_image>0, 1.0, 0)## the first mask (after filtering and thresholding)


**Morphological operations**

In [145]:
import cv2 as cv

noyau = np.ones((3,3),np.uint8)

labeled_image2 =cv.morphologyEx(labeled_image, cv.MORPH_GRADIENT, noyau)
#labeled_image2 = cv.morphologyEx(labeled_image2, cv.MORPH_OPEN, noyau)
labeled_image2 = cv.dilate(labeled_image2,noyau,iterations = 3)


## Region growing algorithm 

In [71]:
def visual_callback_3d(fig=None, plot_each=1):
    """
    Returns a callback than can be passed as the argument `iter_callback`
    of `morphological_geodesic_active_contour` and
    `morphological_chan_vese` for visualizing the evolution
    of the levelsets. Only works for 3D images.
    Parameters
    ----------
    fig : matplotlib.figure.Figure
        Figure where results will be drawn. If not given, a new figure
        will be created.
    plot_each : positive integer
        The plot will be updated once every `plot_each` calls to the callback
        function.
    Returns
    -------
    callback : Python function
        A function that receives a levelset and updates the current plot
        accordingly. This can be passed as the `iter_callback` argument of
        `morphological_geodesic_active_contour` and
        `morphological_chan_vese`.
    """

    from mpl_toolkits.mplot3d import Axes3D
    # PyMCubes package is required for `visual_callback_3d`
    try:
        import mcubes
    except ImportError:
        raise ImportError("PyMCubes is required for 3D `visual_callback_3d`")

    # Prepare the visual environment.
    if fig is None:
        fig = plt.figure()
    fig.clf()
    ax = fig.add_subplot(111, projection='3d')
    plt.pause(0.001)

    counter = [-1]

    def callback(levelset):

        counter[0] += 1
        if (counter[0] % plot_each) != 0:
            return

        if ax.collections:
            del ax.collections[0]

        coords, triangles = mcubes.marching_cubes(levelset, 0.5)
        ax.plot_trisurf(coords[:, 0], coords[:, 1], coords[:, 2],
                        triangles=triangles)
        plt.pause(0.1)

    return callback

**Region growing algorithm**

In [152]:
img = labeled_image2
# Initialization of the level-set.
init_ls = ms.circle_level_set(img.shape, (19, 10, 10), 150)

# Morphological Chan-Vese (or ACWE)
img = ms.morphological_chan_vese(img, iterations=200,
                           init_level_set=init_ls,
                           smoothing=1, lambda1=2, lambda2=1)

img = img.astype('float32')
im = sitk.GetImageFromArray(img)
sitk.Show(im, "Dicom Series")


In [146]:
##first mask (sheetness-like filter)
im = sitk.GetImageFromArray(image_masked)
sitk.Show(im, "Dicom Series")

##morphological operations applied
im = sitk.GetImageFromArray(labeled_image2)
sitk.Show(im, "Dicom Series")

##Region growing algorithm applied (mask inversed later in the code)
img = img.astype('float32')
im = sitk.GetImageFromArray(img)
sitk.Show(im, "Dicom Series")


**Evaluation**

In [150]:
def compute_dice_coefficient(mask_gt, mask_pred):
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN

    volume_intersect = (np.logical_and(mask_gt, mask_pred)).sum()

    return 2 * volume_intersect / volume_sum

In [160]:
score = compute_dice_coefficient(labeled_image2,mask_pred)
print("dice score before region growing algorithm", score)

dice score before region growing algorithm 0.715735964247741


In [161]:
mask_pred2 = np.where(img>0, 0, 1) ##Final mask

score = compute_dice_coefficient(labeled_image2,mask_pred2)
print("dice score after region growing", score)

dice score after region growing 0.8309131829179092


**Final mask visualization**

In [ ]:
im = sitk.GetImageFromArray(mask_pred2)
sitk.Show(im, "Dicom Series")